In [1]:

import os
import json
import pandas as pd
import numpy as np
import pickle as pkl
import seaborn as sns
import gensim
from gensim.test.utils import common_texts
import torch

import torch.nn as nn
import torch.nn.functional as F
# load config
with open('config.json', 'r') as f:
    config = json.load(f)
cwd = os.getcwd()
os.chdir(config['REPODIR'])
import Utils as U
from Corpus import Corpus
os.chdir(cwd)

from collections import Counter, defaultdict
import itertools


from tqdm.auto import tqdm, trange
from collections import Counter
import random
from torch import optim
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

from torch.utils.tensorboard import SummaryWriter

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="dark")
import pickle as pkl
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
import nltk
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate

torch.set_default_dtype(torch.float32)

from torch.utils.data.dataloader import default_collate
from torch.utils.tensorboard import SummaryWriter
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
kwargs = {'num_workers': 4, 'pin_memory': True} \
         if (device == "cuda:0" or device == 'mps') else {}
collate_func = lambda x: tuple(x_.to(device) for x_ in default_collate(x)) \
               if device != "cpu" else default_collate

# +
test_size = 0.2
val_size = 0.2
random_state =699
suffix = "tiny"

# +
with open('embedding_data_final.pkl', 'rb') as f:
    embed = pkl.load(f)

embed_df = pd.DataFrame(embed)

data = U.load_file(f'data_vF_{suffix}.pkl', 'pkl', config['DATADIR'])


In [8]:
embed_df.head()

,author_id,text,join_text,passage_key
0,1082,"[This silence for my sin you did impute,, Whic...","This silence for my sin you did impute, Which ...",PG1041_1675
1,1082,"[Sland’ring creation with a false esteem:, ...",Sland’ring creation with a false esteem: Y...,PG1041_2425
2,1082,"[To me, fair friend, you never can be old,, Fo...","To me, fair friend, you never can be old, For ...",PG1041_2025
3,1082,"[From hands of falsehood, in sure wards of tru...","From hands of falsehood, in sure wards of trus...",PG1041_1075
4,1082,[ To show false Art what beauty was of yore...,To show false Art what beauty was of yore....,PG1041_1425


In [9]:

embed_df = pd.DataFrame(data)

embed_df['join_text'] = embed_df['text'].apply(' '.join)
embed_df = embed_df[['author_id', 'text', 'join_text', 'passage_key']]


In [10]:
import torch
torch.cuda.empty_cache()

In [27]:
len(embed_df)

6530

In [11]:

n_classes = embed_df.author_id.nunique()

from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
y= label_encoder.fit_transform(embed_df['author_id'].to_numpy(dtype='int32').reshape(-1,1))
X = embed_df['passage_key']

X_train, X_test, y_train, y_test = U.train_test_split(X, y, test_size=test_size,
                                                        random_state=random_state,
                                                        stratify=y)

# Split train set into train and validation sets
X_train, X_val, y_train, y_val = U.train_test_split(X_train, y_train, test_size=val_size/(1-test_size),
                                                    random_state=random_state,
                                                    stratify=y_train)



/home/srtoner/.local/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
embed_df['label'] = y

train_set = set(X_train)
val_set = set(X_val)
test_set = set(X_test)

In [14]:
train = embed_df[embed_df['passage_key'].apply(lambda S: S in train_set)]
val = embed_df[embed_df['passage_key'].apply(lambda S: S in val_set)]
test = embed_df[embed_df['passage_key'].apply(lambda S: S in test_set)]

In [15]:
train.to_csv('train.csv', index=False)
val.to_csv('val.csv', index=False)
test.to_csv('test.csv', index=False)

In [16]:
train

,author_id,text,join_text,passage_key,label
1,1082,"[Sland’ring creation with a false esteem:, ...",Sland’ring creation with a false esteem: Y...,PG1041_2425,0
5,1082,"[Giving him aid, my verse astonished., He, nor...","Giving him aid, my verse astonished. He, nor t...",PG1041_1725,0
6,1082,[From where thou art why should I haste me the...,From where thou art why should I haste me then...,PG1041_1125,0
8,1082,"[, CXIII, , Since I left you, mine eye is in m...","CXIII Since I left you, mine eye is in my mi...",PG1041_2175,0
9,1082,"[That tongue that tells the story of thy days,...","That tongue that tells the story of thy days, ...",PG1041_1875,0
...,...,...,...,...,...
6518,1775,"[is difficult to say, since it was only last n...","is difficult to say, since it was only last ni...",PG30445_1389,1
6521,1775,"[Tilsa and Tobene., , 'They are, if I may use ...","Tilsa and Tobene. 'They are, if I may use the...",PG30445_1139,1
6524,1775,"[cave. There was a noise of pots and pans., , ...",cave. There was a noise of pots and pans. 'Is...,PG30445_939,1
6525,1775,[before the anticipation of the meal he would ...,before the anticipation of the meal he would d...,PG30445_1789,1


In [17]:
from datasets import Dataset

In [18]:
ds = {
      'train' :  Dataset.from_csv('train.csv'),
      'val' :  Dataset.from_csv('val.csv'),
      'test' :  Dataset.from_csv('test.csv'),
      }

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/srtoner/.cache/huggingface/datasets/csv/default-3380779cca4b9e40/0.0.0. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/srtoner/.cache/huggingface/datasets/csv/default-ccaaf7e8ee044be1/0.0.0. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/srtoner/.cache/huggingface/datasets/csv/default-1316de57c83f67aa/0.0.0. Subsequent calls will reuse this data.


In [19]:
embed_df.columns

Index(['author_id', 'text', 'join_text', 'passage_key', 'label'], dtype='object')

In [20]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers.integrations import TensorBoardCallback
from transformers.trainer_callback import EarlyStoppingCallback
from torch.utils.data import DataLoader


ds1 = {}

BASE_MODEL = "microsoft/MiniLM-L12-H384-uncased"
# BASE_MODEL = "allenai/longformer-base-4096"
# BASE_MODEL = "lreN5bs16" # Learning Rate 2e-5, batch size 16
LEARNING_RATE = 2e-5
MAX_LENGTH = 510
BATCH_SIZE = 16
EPOCHS = 10

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, 
                                                           num_labels=n_classes,
                                                           ignore_mismatched_sizes=True)

def preprocess_function(examples, test = False):
    # if not test:
    label = examples["label"] 
    examples = tokenizer(examples["join_text"],
                        truncation=True, 
                        padding="max_length",
                        max_length=MAX_LENGTH,
                        return_tensors='pt')
    
    for key in examples:
        examples[key] = examples[key].squeeze(0)
  
    # if not test:
    examples["label"] = torch.IntTensor([label])
    examples = examples.to(device)
    return examples

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
for split in ds:
    ds1[split] = ds[split].map(preprocess_function, 
                                remove_columns=['author_id', 'text', 'passage_key','join_text', 'label'])

    ds1[split].set_format('pt')

Map:   0%|          | 0/3918 [00:00<?, ? examples/s]

Map:   0%|          | 0/1306 [00:00<?, ? examples/s]

Map:   0%|          | 0/1306 [00:00<?, ? examples/s]

In [22]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="output/",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    logging_steps = 1,
    num_train_epochs=EPOCHS,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="F1",
    greater_is_better=True,
    load_best_model_at_end=True,
    weight_decay=0.01,
    report_to = 'tensorboard'
)

early_stop = EarlyStoppingCallback(1, 0.01)
tb = TensorBoardCallback()


from transformers import Trainer
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        loss_fct = nn.functional.cross_entropy
        loss = loss_fct(logits.view(-1, n_classes), labels.view(-1))
        return (loss, outputs) if return_outputs else loss
    

In [23]:
from sklearn.metrics import f1_score as f1s

def compute_metrics_for_classification(eval_pred):
    predictions, labels = eval_pred
    labels = labels.reshape(-1, 1)
    print(labels)
    print(type(predictions))
    
    predicted_class = predictions.argmax(axis=1)
    print(predicted_class)
    f1 = f1s(labels, predicted_class,average = 'weighted')
    print(f"F1: {f1}")
    
    return {"F1": f1}

In [24]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=ds1["train"],
    eval_dataset=ds1["val"],
    compute_metrics=compute_metrics_for_classification,
)

In [25]:
trainer.train()

/home/srtoner/.local/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,0.263200,0.426725,0.828995
2,0.171300,0.294253,0.890969
3,0.389800,0.225322,0.946099
4,0.329300,0.181360,0.961405
5,0.161000,0.137182,0.970806
6,0.043500,0.114216,0.977305
7,0.023800,0.090360,0.980065
8,0.035100,0.103922,0.977139
9,0.014900,0.099015,0.975462
10,0.015400,0.091856,0.979294


[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>
[0 0 0 ... 5 5 5]
F1: 0.8289953762839195
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>
[0 0 0 ... 1 1 1]
F1: 0.8909686761303875
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>
[0 0 0 ... 1 2 3]
F1: 0.9460986714741068
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>
[0 0 0 ... 1 2 3]
F1: 0.9614045845618916
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>
[0 0 0 ... 1 2 3]
F1: 0.9708058328823721
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>
[0 0 0 ... 1 2 3]
F1: 0.977305412308145
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>
[0 0 0 ... 1 2 3]
F1: 0.9800652969654627
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>
[0 0 0 ... 1 2 1]
F1: 0.9771393483961665
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>
[0 0 0 ... 1 2 3]
F1: 0.9754615924365411
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>
[0 0 0 ... 1 2 3]
F1: 0.97929448

TrainOutput(global_step=2450, training_loss=0.19796143624057272, metrics={'train_runtime': 656.4687, 'train_samples_per_second': 59.683, 'train_steps_per_second': 3.732, 'total_flos': 2570996582128800.0, 'train_loss': 0.19796143624057272, 'epoch': 10.0})

In [28]:
trainer.evaluate()

[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>
[0 0 0 ... 1 2 3]
F1: 0.9800652969654627


{'eval_loss': 0.09035958349704742,
 'eval_F1': 0.9800652969654627,
 'eval_runtime': 5.9532,
 'eval_samples_per_second': 219.379,
 'eval_steps_per_second': 13.774,
 'epoch': 10.0}

In [29]:
n_classes

6

In [30]:
prediction = trainer.predict(ds1['test'])

[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>
[0 0 0 ... 1 1 1]
F1: 0.9838759858061444


In [22]:
trainer.save_model(f'hf_model_{suffix}')